# Задание 1.

### Обязательная часть¶

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import json
from datetime import datetime

KEYWORDS = ['JavaScript']

In [64]:
# Обязательная часть

def get_preview_by_keywords (words, URL = 'https://habr.com/ru/all/'):
    res = requests.get(URL)
    soup = BeautifulSoup(res.text, 'html.parser')
    news_blocks = soup.find_all('article', class_ = 'post post_preview')
    
    preview = pd.DataFrame()

    for news_block in news_blocks:
        for key in KEYWORDS:
            if key.lower() in news_block.text.lower():
                date = news_block.find('span', class_ = 'post__time').text
                title = news_block.find('h2', class_ = 'post__title').text
                link = news_block.find('a', class_ = 'post__title_link').get('href')
                all_inf = {'date': date, 'title': title.strip(), 'link': link}
                preview = pd.concat([preview, pd.DataFrame([all_inf])])
          
    preview = preview.drop_duplicates()
    
    return preview

In [65]:
get_preview_by_keywords (KEYWORDS)

date                                              title  \
0  сегодня в 12:00                   Quasar — швейцарский нож для Vue   
0  сегодня в 11:49  Как стать фронтендером? Пошаговый гид в мир фр...   
0  сегодня в 11:36                    Управление памятью в JavaScript   

                                                link  
0    https://habr.com/ru/company/vdsina/blog/525382/  
0  https://habr.com/ru/company/domclick/blog/526244/  
0                   https://habr.com/ru/post/526246/

In [66]:
# Дополнительная часть

def get_preview_and_content_by_keywords (words, URL = 'https://habr.com/ru/all/'):
    res = requests.get(URL)
    soup = BeautifulSoup(res.text, 'html.parser')
    news_blocks = soup.find_all('article', class_ = 'post post_preview')
    
    preview_and_news = pd.DataFrame()

    for news_block in news_blocks:
        for key in KEYWORDS:
            if key.lower() in news_block.text.lower():
                date = news_block.find('span', class_ = 'post__time').text
                title = news_block.find('h2', class_ = 'post__title').text
                link = news_block.find('a', class_ = 'post__title_link').get('href')
                all_inf = {'date': date, 'title': title.strip(), 'link': link}
                preview_and_news = pd.concat([preview_and_news, pd.DataFrame([all_inf])])

    links = list(map(lambda x: x.find('a', class_ = 'post__title_link').get('href'), news_blocks))
                
    for link in links:
        print(link)
        res = requests.get(link)
        soup = BeautifulSoup(res.text, 'html.parser')
        text_content = soup.find('div', class_ = 'post__body post__body_full').text      
        text_content = text_content.replace('\n', ' ')
        text_content = text_content.replace('\xa0', ' ')
        text_content = text_content.replace('\r', ' ')
        for key in KEYWORDS:
            if key in text_content:
                date = soup.find('span', class_ = 'post__time').text
                title = soup.find('h1', class_ = 'post__title post__title_full').text
                all_inf = {'date': date, 'title': title.strip(), 'link': link, 'text_content': text_content}
                preview_and_news = pd.concat([preview_and_news, pd.DataFrame([all_inf])])
        time.sleep(0.2)

    preview_and_news = preview_and_news.drop_duplicates()
    
    return preview_and_news

In [67]:
get_preview_and_content_by_keywords (KEYWORDS)

https://habr.com/ru/company/veeam/blog/525828/
https://habr.com/ru/post/526262/
https://habr.com/ru/company/productsense/blog/526238/
https://habr.com/ru/post/526252/
https://habr.com/ru/company/bizone/blog/526168/
https://habr.com/ru/post/526178/
https://habr.com/ru/company/vdsina/blog/525382/
https://habr.com/ru/post/526160/
https://habr.com/ru/company/domclick/blog/526244/
https://habr.com/ru/post/526248/
https://habr.com/ru/post/526246/
https://habr.com/ru/company/auriga/blog/526090/
https://habr.com/ru/post/513482/
https://habr.com/ru/company/lanit/blog/525778/
https://habr.com/ru/company/near/blog/526232/
https://habr.com/ru/company/croc/blog/526200/
https://habr.com/ru/company/microsoft/blog/526020/
https://habr.com/ru/post/526226/
https://habr.com/ru/company/solarsecurity/blog/526096/
https://habr.com/ru/company/searchinform/blog/526134/


date                                              title  \
0  сегодня в 12:00                   Quasar — швейцарский нож для Vue   
0  сегодня в 11:49  Как стать фронтендером? Пошаговый гид в мир фр...   
0  сегодня в 11:36                    Управление памятью в JavaScript   
0  сегодня в 12:43          Kotlin: язык программирования как продукт   
0  сегодня в 11:49  Как стать фронтендером? Пошаговый гид в мир фр...   
0  сегодня в 11:36                    Управление памятью в JavaScript   

                                                link  \
0    https://habr.com/ru/company/vdsina/blog/525382/   
0  https://habr.com/ru/company/domclick/blog/526244/   
0                   https://habr.com/ru/post/526246/   
0  https://habr.com/ru/company/productsense/blog/...   
0  https://habr.com/ru/company/domclick/blog/526244/   
0                   https://habr.com/ru/post/526246/   

                                        text_content  
0                                                NaN  
0                                                NaN  
0                                                NaN  
0   Язык программирования — это тоже продукт. Он ...  
0      Приветствую всех! Меня зовут Пучнина Анаст...  
0      Доброго времени суток, друзья!   В подавля...

# Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:

EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.

Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'  

TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

In [40]:
EMAIL = ['xxx@x.ru', 'yyy@y.com']
# headers_ = {'Accept': 'application/json, text/plain, */*',
#            'Accept-Encoding': 'gzip, deflate, br',
#            'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
#            'Connection': 'keep-alive',
#            'Content-Length': '33',
#            'Content-Type': 'application/json;charset=utf-8',
#            'DNT': '1',
#            'Host': 'identityprotection.avast.com',
#            'Origin': 'https://www.avast.com',
#            'Referer': 'https://www.avast.com/',
#            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:82.0) Gecko/20100101 Firefox/82.0',
#            'Vaar-Header-App-Product': 'hackcheck-web-avast',
#            'Vaar-Version': '0',
#            'scheme': 'https',
#            'filename': '/v1/web/query/breaches-stats'
# }

headers_ = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:82.0) Gecko/20100101 Firefox/82.0'}

# headers_ = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36',
#            'Request URL': 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
#            }

data_ = {'emailAddresses': 'xxx@x.ru'}


In [41]:
requests.post('https://www.avast.com/hackcheck', data=json.dumps(data_), headers = headers_).text

'<html>\r\n<head><title>405 Not Allowed</title></head>\r\n<body>\r\n<center><h1>405 Not Allowed</h1></center>\r\n<hr><center>nginx</center>\r\n</body>\r\n</html>\r\n'

In [15]:
# Дополнительная часть

def netology():

    news = pd.DataFrame()
    
    NEWSFEED_REQUEST = 'https://api.vk.com/method/wall.get?'

    params = {
        'access_token': '',#здесь нужно вписать токен
        'v': '5.124',
        'domain': 'netology',
        'count': 50,
        'filter': 'owner'
    }
    
    res = requests.get(NEWSFEED_REQUEST, params)
    
    news = pd.DataFrame(res.json()['response']['items'])
    
    news['date_norm'] = news['date'].apply(lambda x: datetime.fromtimestamp(x))
    
    news = news[['date_norm', 'text']]

    return news

In [14]:
netology()

date_norm                                               text
0  2020-10-29 08:33:00  Хэллоуин — время страшных историй, горящих тык...
1  2020-11-03 15:54:00  Кому-то мудборды помогают визуализировать мечт...
2  2020-11-03 09:23:00  Маркетинг переходит в цифровую реальность: рек...
3  2020-11-02 15:36:00  Что важно сделать перед запуском новой диетиче...
4  2020-11-02 08:32:00  🚀 9 ноября стартует бесплатный курс-путеводите...
5  2020-11-01 12:54:00  Несколько выступлений TED для тех, кто ищет ра...
6  2020-10-31 14:52:00  *партнёрский пост* \n \n88% сотрудников считаю...
7  2020-10-31 09:52:00  Как работать с матрицей контента, привлекать т...
8  2020-10-30 15:27:00  Иногда на собеседовании задают вопросы, которы...
9  2020-10-30 08:44:00  Веб-дизайн, геймдизайн, 3D-дизайн — так много ...
10 2020-10-29 15:42:00  Пост в закладки 😁 Скорая помощь для тех, кто м...
11 2020-10-28 14:05:05  Буу! 👻\n\nПриближаются самые страшные выходные...
12 2020-10-28 08:40:00  Кажется, что все известные миру сюжетные повор...
13 2020-10-27 15:42:00  Сценарий видеоролика — не только текст ведущег...
14 2020-10-27 09:18:00  На удалёнке Гугл-документы — незаменимый инстр...
15 2020-10-26 15:54:00  Собрать мудборд, сделать чертёж, создать 3D-мо...
16 2020-10-26 08:34:00  Запустили бесплатный курс по основам контекстн...
17 2020-10-25 13:42:00  С Днём маркетолога, друзья! 🥳\n\nНемного жизне...
18 2020-10-24 13:24:00  В сегодняшней подборке онлайн-мероприятий: кон...
19 2020-10-24 09:49:00  Работой на удалёнке уже никого не удивить: миф...
20 2020-10-23 16:49:00  В эфире #пятничнаябеседка. Здесь мы обсуждаем,...
21 2020-10-23 09:36:00  Погода уже не радует, но это не повод унывать ...
22 2020-10-22 16:42:00  Мечтаем о волшебной таблетке, которая позволит...
23 2020-10-22 09:18:00  Ученые из Гарварда исследовали, что мотивирует...
24 2020-10-21 16:42:00  Время собирать аптечку на осень 2020 💊 \n \nЧт...
25 2020-10-21 09:42:00  🔥 Как выбрать интернет-профессию под себя и ре...
26 2020-10-20 16:42:00  Завести влог на ютубе, начать снимать вайны ил...
27 2020-10-20 10:12:00  Как озвучить драку с ломающимися конечностями ...
28 2020-10-19 16:37:00  Когда нужно обработать картинку, написать текс...
29 2020-10-19 09:24:00  22 октября стартует бесплатный курс «Диджитал-...
30 2020-10-18 14:08:00  Что делать, когда нет идей? 😐\n⠀\nПосмотреть о...
31 2020-10-17 09:24:00  Комментарии, лайки и миллионы подписчиков: соб...
32 2020-10-16 16:08:00  В эфире #пятничнаябеседка. Здесь мы обсуждаем,...
33 2020-10-16 09:24:00  Бесплатных курсов много не бывает: на следующе...
34 2020-10-15 16:44:00  Карьера перестала быть прямой линией из точки ...
35 2020-10-15 09:42:00  Осенью часто хочется замедлиться: плед, кот, к...
36 2020-10-14 16:11:00  2020-й год — год соцсетей. Брендам они помогли...
37 2020-10-14 10:05:05  А у нас конкурс 🎁\n\nРазыгрываем сразу 10 приз...
38 2020-10-13 17:11:00  В мире мессенджеров, пуш-уведомлений и социаль...
39 2020-10-13 10:43:00  «Мне прямо говорили, что в моём возрасте нечег...
40 2020-10-12 16:28:00  Позвонить на горячую линию? 🙅‍♀ \nНаписать на ...
41 2020-10-12 09:24:00  🗓 13 октября стартует бесплатный марафон «Три ...
42 2020-10-11 13:28:00  Собрали книги для тех, кто хочет создать сообщ...
43 2020-10-10 15:31:00  *партнёрский пост*\n\n15 октября в 18:30 пройд...
44 2020-10-10 09:28:00  Камера, мотор 🎬 \nСобрали статьи, которые помо...
45 2020-10-09 16:34:00  В эфире #пятничнаябеседка. Здесь мы обсуждаем,...
46 2020-10-09 09:28:00  Для купальников уже поздно, для пуховиков ещё ...
47 2020-10-08 17:08:00  Поиск работы — волнительный и сложный процесс:...
48 2020-10-08 10:07:00  Выучить китайский язык, сменить работу или зап...
49 2020-10-07 16:44:00  Задачка на внимательность: давайте подготовимс...